In [1]:
import xarray as xr 

In [2]:
vil_ds = xr.open_zarr('../hrrr17x.zarr/')
vil_ds

<xarray.Dataset>
Dimensions:  (id: 5625, lat: 25, lon: 25)
Dimensions without coordinates: id, lat, lon
Data variables:
    VIL      (id, lat, lon) float32 dask.array<chunksize=(1407, 7, 13), meta=np.ndarray>

In [3]:
vil_dr = vil_ds.VIL

# Traffic Setup

In [4]:
# Begin downloading traffic data corresponding to the thunderstorm dates indicated by vil.attrs['date']
from traffic.data import opensky
opensky.username='thinhhoang'
opensky.password='iQ6^yrwe7o3m'
from datetime import datetime, timedelta 
import pytz 

dallas_tz = pytz.timezone('US/Central')
paris_tz = pytz.timezone('Europe/Paris')
utc_tz = pytz.timezone('UTC')

# Some Helper functions to handle datetime

In [5]:
def str_to_datetime_utc(s):
    return utc_tz.localize(datetime.strptime(s, '%Y-%m-%d %H:%M:%S'))

def to_dallas_tz(dt):
    return dt.astimezone(dallas_tz)

def to_paris_tz(dt):
    return dt.astimezone(paris_tz)

def to_utc_tz(dt):
    return dt.astimezone(utc_tz)

def to_string_from_datetime(dt):
    return dt.strftime('%Y-%m-%d %H:%M:%S')

# Contiguous storm catalogue creator

Merge multiple time entries together

In [6]:
def merge_contiguous_storms(datetime_list):
    # Convert to datetime
    datetime_list = [str_to_datetime_utc(dt) for dt in datetime_list]
    
    # Sort the list
    datetime_list.sort()
    
    # Merge contiguous storms
    merged_datetime_list = [] # end of the storm
    merged_datetime_list_begin = [] # beginning of the storm
    
    for dt in datetime_list:
        if len(merged_datetime_list) == 0:
            merged_datetime_list.append(dt)
            merged_datetime_list_begin.append(dt)
        else:
            if dt - merged_datetime_list[-1] > timedelta(hours=1):
                merged_datetime_list.append(dt)
                merged_datetime_list_begin.append(dt)
            else:
                merged_datetime_list[-1] = dt
                
    # Convert back to string
    merged_datetime_list = [to_string_from_datetime(dt) for dt in merged_datetime_list]
    merged_datetime_list_begin = [to_string_from_datetime(dt) for dt in merged_datetime_list_begin]
    
    return merged_datetime_list_begin, merged_datetime_list

In [7]:
storm_catalogue = merge_contiguous_storms(vil_dr.attrs['date']) # merged contiguous storms
storm_catalogue = list(zip(storm_catalogue[0], storm_catalogue[1])) # convert to list of tuples

In [8]:
# create a pandas dataframe to store the data
import pandas as pd

df = pd.DataFrame(storm_catalogue, columns=['begin', 'end'])
# add an index column
df['index'] = df.index
print('There are {} storms in the catalogue'.format(len(df)))

df.to_csv('../hrrr17x_catalogue_contiguous.csv', index=False)

There are 345 storms in the catalogue


# Download data from the first storm

In [9]:
def serialize_and_compress(df, filename, verbose=True):
    df.to_csv(filename, index=False, compression='gzip')
    if verbose:
        print('Saved to {}'.format(filename))

In [10]:
def download_flight_data(from_time_utc, to_time_utc=None, airport='KDFW', verbose = True):
    # If from_time_utc is a string, convert it to datetime (UTC)
    if isinstance(from_time_utc, str):
        from_time = str_to_datetime_utc(from_time_utc)
    else:
        from_time = from_time_utc
        
    if to_time_utc is None:
        to_time = from_time + timedelta(minutes=30)
    else:
        # If to_time_utc is a string, convert it to datetime (UTC)
        if isinstance(to_time_utc, str):
            to_time = str_to_datetime_utc(to_time_utc)
        else:
            to_time = to_time_utc
            
    if verbose:
        print(f'Downloading flight data from {to_string_from_datetime(from_time)} to {to_string_from_datetime(to_time)}')
    
    # Download the flight data from OpenSky
    fl = opensky.history(to_string_from_datetime(from_time), to_string_from_datetime(to_time), arrival_airport=airport)
    
    return fl

In [11]:
def batch_download_flight_data(catalogue, process_id):
    for i, row in catalogue.iterrows():
        try:
            storm_index = row['index']
            #print(row.begin, row.end)
            print('Process id: {} downloading storm '.format(process_id) + str(storm_index))
            #raise NotImplementedError('Need to implement this')
            fl = download_flight_data(row.begin, row.end, verbose=False)
            fl_data = fl.data
            # keep only 
            serialize_and_compress(fl.data, f'/Volumes/Crucial X8/tx/hrrr17x_{storm_index}.csv.gz')
            # print(f'Saved to ../tx/hrrr17x_{storm_index}.csv.gz')
        except Exception as e:
            print(e)
            print(f'Failed to download storm {storm_index}')
            continue

In [12]:
import multiprocess as mp
from multiprocess import Process
import numpy as np

In [13]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')

procs = []

# divide the catalogue into 4 parts

catalogue_chunks = np.array_split(df, 8)

for i, chunk in enumerate(catalogue_chunks):
    p = Process(target=batch_download_flight_data, args=(chunk, i))
    procs.append(p)
    p.start()

for p in procs:
    p.join() # wait for all processes to finish


Process id: 0 downloading storm 0
Process id: 1 downloading storm 44


Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

In [14]:
# !rm -rf ../tx/*